<a href="https://colab.research.google.com/github/reyllama/TTIO/blob/master/04_Rescaling_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.autograd import Variable
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
path = "/content/drive/My Drive/corr3.csv"
df = pd.read_csv(path)
df = df.set_index(keys='stock_id')
df.head()

In [ ]:
dtype = torch.FloatTensor
epochs = 100
emb_dim = 32
embeddings = Variable(torch.randn(emb_dim, len(df)).type(dtype), requires_grad=True)
lr = 0.01
optimizer = optim.Adam([embeddings], lr=lr)
mean = np.abs(df).mean().mean()
coef = 1/mean
loss_track = []

for epoch in range(epochs):
  loss = 0
  for i in range(len(df)):
    v1 = embeddings[:, i].view(emb_dim, -1)
    dist = torch.norm(v1 - embeddings, dim=0).view(len(df), 1)
    corrs = torch.from_numpy(np.abs(coef*np.array(df.iloc[:, i]))-1).view(1, len(df)).type(torch.FloatTensor)
    loss += torch.mm(corrs, dist)/2
  loss_track.append(loss.item())
  if (epoch+1) % 5 == 0:
    print("{0}th epoch in process".format(epoch+1))
    print('running loss: {}'.format(loss.item()))
    print()

  optimizer.zero_grad()
  loss.backward(retain_graph=False)
  optimizer.step()

  with torch.no_grad():
    for element in embeddings:
      element.clamp_(0,1)

embeddings[:, 1]  

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16,9))
plt.plot(list(range(epochs)), loss_track)
plt.show()

In [ ]:
embeddings

tensor([[0.5709, 0.5711, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
        [0.5379, 0.5357, 0.0000,  ..., 1.0000, 0.0000, 1.0000],
        [0.5088, 0.5078, 1.0000,  ..., 1.0000, 0.0000, 0.0000],
        ...,
        [0.5198, 0.5197, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [0.5329, 0.5329, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.5397, 0.5382, 0.0000,  ..., 0.0000, 1.0000, 0.0000]],
       requires_grad=True)

## Rescaling Network

In [ ]:
data = pd.read_csv("/content/drive/My Drive/stocks.csv")
data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,stock_id,name,date,opening_price,highest_price,lowest_price,closing_price,trading_volume
0,000020,동화약품,2016-01-04,8130,8150,7920,8140,281440
1,000020,동화약품,2016-01-05,8040,8250,8000,8190,243179
2,000020,동화약품,2016-01-06,8200,8590,8110,8550,609906
3,000020,동화약품,2016-01-07,8470,8690,8190,8380,704752
4,000020,동화약품,2016-01-08,8210,8900,8130,8770,802330


In [ ]:
data['KLength'] = data['closing_price'] - data['opening_price']
data['KUpperLength'] = data['highest_price'] - data[['opening_price', 'closing_price']].max(axis=1)
data['temp'] = data['opening_price']-data['lowest_price']
data['KLowerLength'] = data[['closing_price', 'temp']].min(axis=1)
data['return'] = data['closing_price'].diff()
data = data.drop(columns=['temp'])
data['stock_id'] = data['stock_id'].astype(str)
## 첫날 빼줘야됨 (종목 바뀔때 섞여들어갔다)
data.head()

In [ ]:
def year_parser(column):
  return int(column[0:4])

def month_parser(column):
  return int(column[5:7])

data['year'] = data['date'].apply(year_parser)
data['month'] = data['date'].apply(month_parser)

data = data[(data['year']==2016)&(data['month']<=6)]
data = data[data['stock_id'].isin(df.columns)]
# data = data.fillna(0)

KL = data.pivot(index='date', columns='stock_id', values='KLength')
R = data.pivot(index='date', columns='stock_id', values='return')

KL.shape

(121, 1863)

In [ ]:
print(KL.shape, R.shape)

(121, 2018) (121, 2018)


In [ ]:
KL = KL.iloc[:-1, ]
R = R.iloc[1:, ]

In [ ]:
dtype = torch.FloatTensor
weights = Variable(torch.rand(1, 32).type(dtype), requires_grad=True)
lr = 0.005
optimizer = optim.Adam([weights], lr=lr)
epochs = 3000
indic = np.array(KL)
returns = np.array(R)

for epoch in range(epochs):
  loss = 0

  rescale_fc = torch.exp(torch.mm(weights, embeddings)) / torch.sum(torch.exp(torch.mm(weights, embeddings)))
  opt_ind = torch.from_numpy(indic) * rescale_fc # element-wise
  # print('opt_ind: ', opt_ind)
  mean_ind = torch.mean(opt_ind, axis=0)
  # print('mean_ind: ', mean_ind)
  mean_ret = torch.mean(torch.from_numpy(returns), axis=0)
  # print('mean_ret: ', mean_ret)

  vx = opt_ind - mean_ind
  vy = torch.from_numpy(returns) - mean_ret
  # print('vx: ', vx)
  # print('vy: ', vy)
  loss = -torch.abs(torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2))))
  # print('loss: ', loss)

  if (epoch+1) % 100 == 0:
    print("{0}th epoch in process".format(epoch+1))
    print('running loss: {}'.format(loss.item()))
    print()

  optimizer.zero_grad()
  loss.backward(retain_graph=False)
  optimizer.step()

  with torch.no_grad():
    for weight in weights:
      weight.clamp_(0,1)

weights

### Comparison: Raw KLength vs Rescaled KLength

In [ ]:
mean_KL = np.mean(indic, axis=0)
mean_return = np.mean(returns, axis=0)

vx = indic - mean_KL
vy = returns - mean_return

corr1 = np.abs(np.sum(vx*vy) / (np.sqrt(np.sum(vx ** 2)) * np.sqrt(np.sum(vy ** 2))))

rescale_fc = torch.exp(torch.mm(weights, embeddings)) / torch.sum(torch.exp(torch.mm(weights, embeddings)))
opt_ind = torch.from_numpy(indic) * rescale_fc # element-wise
mean_ind = torch.mean(opt_ind, axis=0)
mean_ret = torch.mean(torch.from_numpy(returns), axis=0)

wx = opt_ind - mean_ind
wy = torch.from_numpy(returns) - mean_ret
corr2 = torch.abs(torch.sum(wx * wy) / (torch.sqrt(torch.sum(wx ** 2)) * torch.sqrt(torch.sum(wy ** 2))))

In [ ]:
print("Correlation of Raw vs Recaled: {0:.4f} vs {1:.4f}".format(corr1, corr2))

Correlation of Raw vs Recaled: 0.2428 vs 0.2473


In [ ]:
!pip install ta

  Created wheel for ta: filename=ta-0.5.25-cp36-none-any.whl size=24880 sha256=570a505dd5ce4b2c21a3fd9fc98c7c94b2e2c03ca52682f343b7a5f830117ce6
  Stored in directory: /root/.cache/pip/wheels/2e/93/b7/cf649194508e53cee4145ffb949e9f26877a5a8dd12db9ed5b
Successfully built ta


In [ ]:
import ta

def bias(close=None, m=7):
  return close - close.rolling(m).mean()

soda = data.copy()
soda['EMA'] = ta.trend.ema_indicator(close=data['closing_price'])
soda['MACD'] = ta.trend.macd(close=data['closing_price'])
soda['ROC'] = ta.momentum.roc(close=data['closing_price'])
soda['BIAS'] = bias(soda['closing_price'])
soda.iloc[3:28]

In [ ]:
path = "/content/drive/My Drive/stocks_merged.csv"
merg = pd.read_csv(path)
merg.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,stock_id,name,date,opening_price,highest_price,lowest_price,closing_price,trading_volume,industry
0,0,20,동화약품,2016-01-04,8130,8150,7920,8140,281440,의약품 제조업
1,1,20,동화약품,2016-01-05,8040,8250,8000,8190,243179,의약품 제조업
2,2,20,동화약품,2016-01-06,8200,8590,8110,8550,609906,의약품 제조업
3,3,20,동화약품,2016-01-07,8470,8690,8190,8380,704752,의약품 제조업
4,4,20,동화약품,2016-01-08,8210,8900,8130,8770,802330,의약품 제조업


In [ ]:
merg['industry'].unique().shape

(148,)

In [ ]:
merg['industry'].unique()

array(['의약품 제조업', '그외 기타 운송장비 제조업', '종합 소매업', '보험업', '기타 금융업', nan,
       '알코올음료 제조업', '도로 화물 운송업', '회사 본부 및 경영 컨설팅 서비스업', '토목 건설업',
       '통신 및 방송 장비 제조업', '자동차 신품 부품 제조업', '기타 전문 도매업', '내화, 비내화 요업제품 제조업',
       '특수 목적용 기계 제조업', '절연선 및 케이블 제조업', '육상 여객 운송업', '반도체 제조업',
       '1차 비철금속 제조업', '상품 종합 도매업', '기타 화학제품 제조업',
       '시멘트, 석회, 플라스터 및 그 제품 제조업', '방적 및 가공사 제조업', '가전제품 및 정보통신장비 소매업',
       '펄프, 종이 및 판지 제조업', '직물직조 및 직물제품 제조업', '1차 철강 제조업', '상품 중개업',
       '곡물가공품, 전분 및 전분제품 제조업', '전구 및 조명장치 제조업', '금융 지원 서비스업',
       '기초 화학물질 제조업', '봉제의복 제조업', '비료, 농약 및 살균, 살충제 제조업',
       '기타 비금속 광물제품 제조업', '기타 식품 제조업', '건물 건설업',
       '건축기술, 엔지니어링 및 관련 기술 서비스업', '가죽, 가방 및 유사제품 제조업',
       '측정, 시험, 항해, 제어 및 기타 정밀기기 제조업; 광학기기 제외', '낙농제품 및 식용빙과류 제조업',
       '골판지, 종이 상자 및 종이용기 제조업', '고무제품 제조업', '영화, 비디오물, 방송프로그램 제작 및 배급업',
       '악기 제조업', '기계장비 및 관련 물품 도매업', '나무제품 제조업', '석유 정제품 제조업',
       '기타 운송관련 서비스업', '화학섬유 제조업', '해상 운송업', '동물용 사료 및 조제식품 제조업',
       '연료용 가스 제조 및 배관공급업', '항공 여객 운송업', 

### Modulate

In [ ]:
path = "/content/drive/My Drive/corr1.csv"
df = pd.read_csv(path)
df = df.set_index(keys='stock_id')
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,000020,000040,000050,000060,000070,000075,000080,000087,000100,000105,000120,000140,000145,000150,000155,000157,000180,000210,000215,000220,000225,000227,000230,000240,000250,000270,000300,000320,000325,000370,000390,000400,000430,000440,000480,000490,000500,000520,000540,000545,...,95190,95270,95340,95500,95570,95610,95660,95700,95720,95910,96040,96240,96300,96350,96530,96610,96630,96640,96760,96770,96775,96870,97230,97520,97780,97800,97870,97950,97955,98120,98460,98660,99190,99220,99320,99340,99350,99410,99440,99520
stock_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000020,1.000000,0.221514,-0.037755,-0.021159,0.230919,0.259171,0.176753,0.195048,0.215654,0.175516,0.208703,0.139837,0.071031,0.116629,0.085471,0.165149,0.124488,0.078928,0.049496,0.537236,0.512802,0.303855,0.483129,0.017378,0.482491,-0.072407,0.017935,0.133918,0.041311,0.148041,0.110374,0.294617,0.040400,0.118770,-0.026300,0.162403,0.135262,0.439909,0.150139,0.018031,...,-0.019455,0.068774,0.035582,0.072378,0.177056,0.094984,0.257213,0.118036,0.061175,0.049229,-0.120927,0.027107,0.229460,0.137070,0.333912,0.253663,-0.004647,0.073918,0.436779,0.070719,0.038662,-0.009037,-0.067703,0.160585,0.272692,0.061822,0.225241,0.039099,-0.051827,0.112372,0.146558,0.079731,0.119611,0.284219,0.065122,-0.193870,-0.055491,0.170469,0.298094,0.101488
000040,0.221514,1.000000,-0.084175,0.125700,0.048086,0.144288,-0.026518,0.139692,0.059997,0.230597,0.052476,0.017729,0.078381,0.209605,0.249582,0.245019,0.194707,0.179482,0.216908,0.249301,0.259880,0.179810,0.216462,0.096105,0.369667,0.131545,0.340488,0.217145,-0.060981,0.220581,0.082751,0.271047,0.300337,0.120322,0.053789,0.174838,0.239307,0.215961,0.197419,0.051451,...,0.268494,0.198407,0.206406,0.414345,0.101782,0.059960,0.227716,0.063983,0.198506,0.071940,-0.059623,0.270323,0.338363,0.194589,0.144394,0.157667,0.114744,0.003995,0.296121,0.238160,0.193186,0.199345,0.176108,0.322742,0.215993,0.116542,0.265938,-0.006009,-0.078675,0.207658,0.136085,0.270371,0.137788,0.219789,0.093931,-0.145495,-0.084699,0.106058,0.374286,0.271006
000050,-0.037755,-0.084175,1.000000,-0.026910,0.194396,0.145192,0.064836,0.065237,0.176480,0.184645,0.139954,0.074693,-0.034289,0.035887,0.030783,-0.029888,0.065450,-0.033895,0.003912,-0.018252,-0.050283,-0.105596,0.237094,0.159021,0.055958,-0.161801,0.105222,0.147823,0.026685,0.059532,0.117234,0.064063,0.016387,-0.015661,0.142240,0.177168,-0.066760,0.106400,0.167397,0.047480,...,0.185434,0.091975,0.040803,0.066808,-0.015634,-0.007347,0.165473,0.041894,0.193574,0.004915,0.085066,0.082473,-0.012954,-0.084595,0.112413,0.189654,0.146540,-0.075025,-0.006074,0.014898,0.053050,0.100744,0.187599,0.086372,-0.100047,-0.065204,-0.016283,0.050333,0.018721,0.032135,0.194629,-0.034360,0.241911,-0.094872,0.076623,0.101066,-0.186058,0.078683,0.081244,-0.111201
000060,-0.021159,0.125700,-0.026910,1.000000,0.178085,0.176415,0.025524,0.103304,0.094681,0.087511,0.031110,0.115337,0.369805,0.158645,0.135941,0.104001,0.026601,0.296269,0.305429,0.199097,0.127579,0.156409,-0.046473,0.031214,0.126763,0.006368,0.272023,0.066901,0.042691,0.142887,0.253146,0.276613,0.158297,0.213465,0.280961,0.082161,0.054212,0.177353,0.171967,0.140361,...,-0.016416,0.065066,0.113294,0.249084,0.048593,0.146570,0.088966,0.125445,0.113970,0.061257,-0.018510,0.115044,0.178463,0.144524,0.098360,-0.042493,0.190262,0.153143,0.129936,0.167621,0.240689,0.047773,0.158049,0.017242,0.192288,0.098559,0.234262,0.172853,0.153594,0.259511,0.211723,-0.002448,0.289181,0.101877,0.057986,0.118551,0.005246,0.018182,0.133223,0.102192
000070,0.230919,0.048086,0.194396,0.178085,1.000000,0.658519,0.103320,0.213751,0.344049,0.187832,0.215465,0.226021,0.029073,0.179910,0.198224,0.239788,0.111998,0.209418,0.262103,0.363887,0.244465,0.167201,0.351404,0.071581,0.286885,-0.196303,0.235813,0.029320,0.166511,0.166894,0.243878,0.293949,0.183960,0.070419,0.150535,0.236201,0.186679,0.268939,0.107885,0.221739,...,0.086512,0.061532,0.316266,0.349720,0

In [ ]:
data = pd.read_csv("/content/drive/My Drive/stocks.csv")
data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,stock_id,name,date,opening_price,highest_price,lowest_price,closing_price,trading_volume
0,000020,동화약품,2016-01-04,8130,8150,7920,8140,281440
1,000020,동화약품,2016-01-05,8040,8250,8000,8190,243179
2,000020,동화약품,2016-01-06,8200,8590,8110,8550,609906
3,000020,동화약품,2016-01-07,8470,8690,8190,8380,704752
4,000020,동화약품,2016-01-08,8210,8900,8130,8770,802330


In [ ]:
def get_embeddings(corr_df=df, n_epochs=100, dim=32, lr=0.01):
  dtype = torch.FloatTensor
  epochs = n_epochs
  emb_dim = dim
  embeddings = Variable(torch.randn(emb_dim, len(df)).type(dtype), requires_grad=True)
  lr = lr
  optimizer = optim.Adam([embeddings], lr=lr)
  mean = np.abs(df).mean().mean()
  coef = 2 # Hyper Parameter 변경
  loss_track = []

  for epoch in range(epochs):
    loss = 0
    for i in range(len(df)):
      v1 = embeddings[:, i].view(emb_dim, -1)
      dist = torch.norm(v1 - embeddings, dim=0).view(len(df), 1)
      corrs = torch.from_numpy(np.abs(coef*np.array(df.iloc[:, i]))-1).view(1, len(df)).type(torch.FloatTensor)
      loss += torch.mm(corrs, dist)/2
    loss_track.append(loss.item())
    if (epoch+1) % 10 == 0:
      print("{0}th epoch in process".format(epoch+1))
      print('running loss: {}'.format(loss.item()))
      print()

    optimizer.zero_grad()
    loss.backward(retain_graph=False)
    optimizer.step()

    with torch.no_grad():
      for element in embeddings:
        element.clamp_(0,1)

  return embeddings

In [ ]:
def get_embeddings(corr_df=df, n_epochs=100, dim=32, lr=0.01):
  dtype = torch.FloatTensor
  epochs = n_epochs
  emb_dim = dim
  embeddings = Variable(torch.randn(emb_dim, len(df)).type(dtype), requires_grad=True)
  lr = lr
  optimizer = optim.Adam([embeddings], lr=lr)
  mean = np.abs(df).mean().mean()
  coef = 2 # Hyper Parameter 변경
  loss_track = []

  for epoch in range(epochs):
    loss = 0
    for i in range(len(df)):
      v1 = embeddings[:, i].view(emb_dim, -1)
      dist = torch.norm(v1 - embeddings, dim=0).view(len(df), 1)
      corrs = torch.from_numpy(np.abs(coef*np.array(df.iloc[:, i]))-1).view(1, len(df)).type(torch.FloatTensor)
      loss += torch.mm(corrs, dist)/2
    loss_track.append(loss.item())
    if (epoch+1) % 10 == 0:
      print("{0}th epoch in process".format(epoch+1))
      print('running loss: {}'.format(loss.item()))
      print()

    optimizer.zero_grad()
    loss.backward(retain_graph=False)
    optimizer.step()

    with torch.no_grad():
      for element in embeddings:
        element.clamp_(0,1)

  return embeddings

In [ ]:
import ta

def bias(close=None, m=7):
  return close - close.rolling(m).mean()

def preprocessing(data=data, year=2016, start_month=7, end_month = 9, corr_df = df):

  data['stock_id'] = data['stock_id'].astype(str)
  data['year'] = data['date'].apply(lambda x: int(x[0:4]))
  data['month'] = data['date'].apply(lambda x: int(x[5:7]))

  data = data[(data['year']==year)&(data['month']<=end_month)&(data['month']>=start_month)]
  data = data[data['stock_id'].isin(df.columns)]

  data['KLength'] = data['closing_price'] - data['opening_price']
  data['KUpperLength'] = data['highest_price'] - data[['opening_price', 'closing_price']].max(axis=1)
  data['temp'] = data['opening_price']-data['lowest_price']
  data['KLowerLength'] = data[['closing_price', 'temp']].min(axis=1)
  data['return'] = data['closing_price'].diff()
  data = data.drop(columns=['temp'])

  data['EMA'] = ta.trend.ema_indicator(close=data['closing_price'])
  data['MACD'] = ta.trend.macd(close=data['closing_price'])
  data['ROC'] = ta.momentum.roc(close=data['closing_price'])
  data['BIAS'] = bias(data['closing_price'])
  

  return data

In [ ]:
def weight_optimize(data=data, indicator='KLength', lr=0.005, n_epochs=3000, embeddings=None):
  KL = data.pivot(index='date', columns='stock_id', values=indicator)
  R = data.pivot(index='date', columns='stock_id', values='return')

  if indicator in ['KLength', 'KUpperLength', 'KLowerLength']:
    KL = KL.iloc[:-1, ]
    R = R.iloc[1:, ]
  elif indicator == 'EMA':
    KL = KL.iloc[11:-1, ]
    R = R.iloc[12:, ]
  elif indicator == 'MACD':
    KL = KL.iloc[25:-1, ]
    R = R.iloc[26:, ]
  elif indicator == 'ROC':
    KL = KL.iloc[12:-1, ]
    R = R.iloc[13:, ]
  elif indicator == 'BIAS':
    KL = KL.iloc[6:-1, ]
    R = R.iloc[7:, ]
  else:
    print('Unknown Indicator!')
    return None

  dtype = torch.FloatTensor
  weights = Variable(2*torch.rand(1, 32).type(dtype)-1, requires_grad=True)
  lr = lr
  optimizer = optim.Adam([weights], lr=lr)
  epochs = n_epochs
  indic = np.array(KL)
  returns = np.array(R)

  for epoch in range(epochs):
    loss = 0

    rescale_fc = torch.exp(torch.mm(weights, embeddings)) / torch.sum(torch.exp(torch.mm(weights, embeddings)))
    opt_ind = torch.from_numpy(indic) * rescale_fc # element-wise
  # print('opt_ind: ', opt_ind)
    mean_ind = torch.mean(opt_ind, axis=0)
  # print('mean_ind: ', mean_ind)
    mean_ret = torch.mean(torch.from_numpy(returns), axis=0)
  # print('mean_ret: ', mean_ret)

    vx = opt_ind - mean_ind
    vy = torch.from_numpy(returns) - mean_ret
  # print('vx: ', vx)
  # print('vy: ', vy)
    loss = -torch.abs(torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2))))
  # print('loss: ', loss)

    if (epoch+1) % 1000 == 0:
      print("{0}th epoch in process".format(epoch+1))
      print('running loss: {:.4f}'.format(loss.item()))
      print()

    optimizer.zero_grad()
    loss.backward(retain_graph=False)
    optimizer.step()

    with torch.no_grad():
      for weight in weights:
        weight.clamp_(0,1)

  mean_KL = np.mean(indic, axis=0)
  mean_return = np.mean(returns, axis=0)

  vx = indic - mean_KL
  vy = returns - mean_return

  corr1 = np.abs(np.sum(vx*vy) / (np.sqrt(np.sum(vx ** 2)) * np.sqrt(np.sum(vy ** 2))))

  rescale_fc = torch.exp(torch.mm(weights, embeddings)) / torch.sum(torch.exp(torch.mm(weights, embeddings)))
  opt_ind = torch.from_numpy(indic) * rescale_fc # element-wise
  mean_ind = torch.mean(opt_ind, axis=0)
  mean_ret = torch.mean(torch.from_numpy(returns), axis=0)

  wx = opt_ind - mean_ind
  wy = torch.from_numpy(returns) - mean_ret
  corr2 = torch.abs(torch.sum(wx * wy) / (torch.sqrt(torch.sum(wx ** 2)) * torch.sqrt(torch.sum(wy ** 2))))

  print("Correlation of Raw vs Recaled: {0:.4f} vs {1:.4f}".format(corr1, corr2))

  return weights

In [ ]:
def test(data=None, weights=None, indicator='KLength'):
  KL = data.pivot(index='date', columns='stock_id', values=indicator)
  R = data.pivot(index='date', columns='stock_id', values='return')

  if indicator in ['KLength', 'KUpperLength', 'KLowerLength']:
    KL = KL.iloc[:-1, ]
    R = R.iloc[1:, ]
  elif indicator == 'EMA':
    KL = KL.iloc[11:-1, ]
    R = R.iloc[12:, ]
  elif indicator == 'MACD':
    KL = KL.iloc[25:-1, ]
    R = R.iloc[26:, ]
  elif indicator == 'ROC':
    KL = KL.iloc[12:-1, ]
    R = R.iloc[13:, ]
  elif indicator == 'BIAS':
    KL = KL.iloc[6:-1, ]
    R = R.iloc[7:, ]
  else:
    print('Unknown Indicator!')
    return None

  indic = np.array(KL)
  returns = np.array(R)
  mean_KL = np.mean(indic, axis=0)
  mean_return = np.mean(returns, axis=0)
  vx = indic - mean_KL
  vy = returns - mean_return
  corr1 = np.abs(np.sum(vx*vy) / (np.sqrt(np.sum(vx ** 2)) * np.sqrt(np.sum(vy ** 2))))

  rescale_fc = torch.exp(torch.mm(weights, embeddings)) / torch.sum(torch.exp(torch.mm(weights, embeddings)))
  opt_ind = torch.from_numpy(indic) * rescale_fc
  mean_ind = torch.mean(opt_ind, axis=0)

  wx = opt_ind - mean_ind
  wy = torch.from_numpy(vy)
  corr2 = torch.abs(torch.sum(wx * wy) / (torch.sqrt(torch.sum(wx ** 2)) * torch.sqrt(torch.sum(wy ** 2))))

  print("Correlation of Raw vs Recaled: {0:.4f} vs {1:.4f}".format(corr1, corr2))

In [ ]:
#### 각 종목별로 Correlation 구해서 평균 내는 방식을 취하면, Correlation Coefficient가 Unit Invariant 하므로 어떤 Weight 곱해서 Rescaling해도 결과가 달라지지 않는다.

KL = test_df.pivot(index='date', columns='stock_id', values='KLength')
R = test_df.pivot(index='date', columns='stock_id', values='return')
KL = KL.iloc[:-1, ]
R = R.iloc[1:, ]
indic = np.array(KL)
returns = np.array(R)
mean_KL = np.mean(indic, axis=0)
mean_return = np.mean(returns, axis=0)
vx = indic - mean_KL
vy = returns - mean_return
corr1 = np.abs(np.sum(vx*vy, axis=0) / (np.sqrt(np.sum(vx ** 2, axis=0)+1e-10) * np.sqrt(np.sum(vy ** 2, axis=0)+1e-10)))

# print(corr1.isnan().sum())
# corr1
rescale_fc = torch.exp(torch.mm(inds, embeddings)) / torch.sum(torch.exp(torch.mm(inds, embeddings)))
opt_ind = torch.from_numpy(indic) * rescale_fc
mean_ind = torch.mean(opt_ind, axis=0)

wx = opt_ind - mean_ind
wy = torch.from_numpy(vy)
corr2 = torch.abs(torch.sum(wx * wy, axis=0) / (torch.sqrt(torch.sum(wx ** 2, axis=0)+1e-10) * torch.sqrt(torch.sum(wy ** 2, axis=0)+1e-10)))

print("Correlation of Raw vs Recaled: {0:.4f} vs {1:.4f}".format(corr1.mean(), corr2.mean()))

#### 내일 할 일
#### 여러 가지 Technical Indicator들의 선형결합으로 예측력 시험하기
#### 즉, 각 종목에 대한 각 Indicator의 Weight를 학습해서, 이를 이용해서 여러 Indicator 들을 결합, 이 값과 리턴 사이의 상관관계 따져보기

#### EX: I_i = weight_1 * I_i,1 + weight_2 * I_i,2 + weight_3 * I_i,3

Correlation of Raw vs Recaled: 0.1197 vs 0.1197


In [ ]:
embeddings = get_embeddings(corr_df=df)
train_df = preprocessing(data=data, year=2016, start_month=1, end_month=6, corr_df=df)
test_df = preprocessing(data=data, year=2016, start_month=7, end_month=9, corr_df=df)

10th epoch in process
running loss: -4036878.75

20th epoch in process
running loss: -4236393.0

30th epoch in process
running loss: -4416008.5

40th epoch in process
running loss: -4566423.0

50th epoch in process
running loss: -4678634.0

60th epoch in process
running loss: -4725080.5

70th epoch in process
running loss: -4728984.0

80th epoch in process
running loss: -4729123.5

90th epoch in process
running loss: -4729123.5

100th epoch in process
running loss: -4729123.5



(2108533, 14)


In [ ]:
inds = weight_optimize(indicator='KUpperLength', data=train_df, embeddings=embeddings, n_epochs=1000)
test(data=test_df, weights=inds, indicator='KUpperLength')

1000th epoch in process
running loss: -0.2510

Correlation of Raw vs Recaled: 0.2470 vs 0.2510
Correlation of Raw vs Recaled: 0.0439 vs 0.0407


In [ ]:
inds = weight_optimize(indicator='KLength', data=train_df, embeddings=embeddings, n_epochs=3000)
test(data=test_df, weights=inds, indicator='KLength')

1000th epoch in process
running loss: -0.2464

2000th epoch in process
running loss: -0.2484

3000th epoch in process
running loss: -0.2485

Correlation of Raw vs Recaled: 0.2437 vs 0.2485
Correlation of Raw vs Recaled: 0.0216 vs 0.0245


In [ ]:
inds = weight_optimize(indicator='KLowerLength', lr=0.005, n_epochs=500, embeddings=embeddings, data=train_df)
test(data=test_df, weights=inds, indicator='KLowerLength')

Correlation of Raw vs Recaled: 0.0306 vs 0.0663
Correlation of Raw vs Recaled: 0.0374 vs 0.0209


In [ ]:
inds = weight_optimize(indicator='EMA', data=train_df, embeddings=embeddings, n_epochs=1000)
test(data=test_df, weights=inds, indicator='EMA')

1000th epoch in process
running loss: -0.0754

Correlation of Raw vs Recaled: 0.0665 vs 0.0754
Correlation of Raw vs Recaled: 0.1663 vs 0.1417


In [ ]:
inds = weight_optimize(indicator='MACD', data=train_df, embeddings=embeddings, n_epochs=2000)
test(data=test_df, weights=inds, indicator='MACD')

1000th epoch in process
running loss: -0.0441

2000th epoch in process
running loss: -0.0441

Correlation of Raw vs Recaled: 0.0346 vs 0.0441
Correlation of Raw vs Recaled: 0.0376 vs 0.0429


In [ ]:
inds = weight_optimize(indicator='ROC', data=train_df, embeddings=embeddings, n_epochs=1500)
test(data=test_df, weights=inds, indicator='ROC')

1000th epoch in process
running loss: -0.0260

Correlation of Raw vs Recaled: 0.0073 vs 0.0260
Correlation of Raw vs Recaled: 0.0224 vs 0.0081


In [ ]:
inds = weight_optimize(indicator='BIAS', data=train_df, embeddings=embeddings, n_epochs=1000)
test(data=test_df, weights=inds, indicator='BIAS')

1000th epoch in process
running loss: -0.0338

Correlation of Raw vs Recaled: 0.0212 vs 0.0338
Correlation of Raw vs Recaled: 0.0431 vs 0.0930
